# Crear un Folleto de el sitio web completo

Crear un programa que cree un folleto para una compañia y este pueda ser usado para clientes, inversores o reclutadores potenciales.



In [38]:
import requests
import json
import re
from collections import Counter
from urllib.parse import urlparse
from typing import List
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from langchain_ollama.llms import OllamaLLM


In [39]:
# Instanciar LLM
model = OllamaLLM(model="qwen2.5-coder:0.5b")

In [40]:
# Misma clase que archivo de la carpeta intro, con una funcionalidad extra
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""

        # Creación de una lista que contiene todos los links, utilizando find_all para obtener todas las equitetas<a href=""></a> de html
        links = [link.get('href') for link in soup.find_all('a')]

        # asignando dicha lista al atributo links de cada instancia de clase, solo incluyendo aquellos que empiecen con `/` o `https`
        links = [link for link in links if link and (link.startswith('/') or link.startswith('https'))]
        print(len(links))
        for i, elem in enumerate(links):
            if elem.startswith('/'):
                links[i] = url + elem
        self.links = links
        print(len(self.links))


    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [41]:
langcahin = Website("https://www.langchain.com/")
langcahin.links

94
56


['https://www.langchain.com//',
 'https://www.langchain.com//langchain',
 'https://www.langchain.com//langsmith',
 'https://www.langchain.com//langgraph',
 'https://www.langchain.com//retrieval',
 'https://www.langchain.com//agents',
 'https://www.langchain.com//evaluation',
 'https://www.langchain.com//customers',
 'https://www.langchain.com//community',
 'https://www.langchain.com//experts',
 'https://www.langchain.com//testing-guide-ebook',
 'https://www.langchain.com//stateofaiagents',
 'https://www.langchain.com//breakoutagents',
 'https://www.langchain.com//about',
 'https://www.langchain.com//careers',
 'https://www.langchain.com//pricing-langsmith',
 'https://www.langchain.com//pricing-langgraph-platform',
 'https://www.langchain.com//contact-sales',
 'https://www.langchain.com//',
 'https://www.langchain.com//langchain',
 'https://www.langchain.com//langsmith',
 'https://www.langchain.com//langgraph',
 'https://www.langchain.com//retrieval',
 'https://www.langchain.com//agents

### Seleccion de links por medio de codigo

El definir una manera de 'filtrar' los links mas relevantes para la construccion del folleto, utilizando solo codigo, es posible, sin embargo no tan sencillo, incluso dependiendo de la estructura del sitio web, el rendimiento puede variar

La manera mas sencilla de realizar esto es por medio de un 'filtro' el cual por medio de la aparacion de las palabra clave mas comunes, o que nos interesen, en la estructura de los URL, tome los resultados mas relevantes

In [25]:
def filter_relevant_links(url):
    link_list = Website(url).links
    # Define common keywords in the URL
    print(f"from {len(link_list)} urls")
    common_keywords = ['company', 'about', 'contact', 'support', 'team', 'careers']
    # Calculate a relevance score based on the number of common keywords found in the URL path
    scores = {link: sum(1 for keyword in common_keywords if re.search(r'\b' + keyword + r'\b', urlparse(link).path)) for link in link_list}
    print(scores)
    # Sort links by their relevance score and filter out top-scoring links until we reach a certain number of links
    num_relevant_links = int(len(link_list) * 0.2)  # Calculate the percentage of relevant links to keep
    sorted_scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    filtered_links = [link for link, score in sorted_scores[:num_relevant_links]]
    print(f"to {len(filtered_links)} urls")
    return filtered_links

In [26]:
print(filter_relevant_links('https://www.langchain.com'))

from 94 urls
{'/': 0, '/langchain': 0, '/langsmith': 0, '/langgraph': 0, '/retrieval': 0, '/agents': 0, '/evaluation': 0, 'https://blog.langchain.dev/': 0, '/customers': 0, 'https://academy.langchain.com/': 0, '/community': 0, '/experts': 0, 'https://changelog.langchain.com/': 0, '/testing-guide-ebook': 0, '/stateofaiagents': 0, '/breakoutagents': 0, 'https://python.langchain.com/docs/introduction/': 0, 'https://docs.smith.langchain.com/': 0, 'https://langchain-ai.github.io/langgraph/tutorials/introduction/': 0, 'https://js.langchain.com/docs/introduction/': 0, 'https://langchain-ai.github.io/langgraphjs/tutorials/quickstart/': 0, '/about': 1, '/careers': 1, '/pricing-langsmith': 0, '/pricing-langgraph-platform': 0, '/contact-sales': 1, 'https://smith.langchain.com/': 0, 'https://blog.langchain.dev/ally-financial-collaborates-with-langchain-to-deliver-critical-coding-module-to-mask-personal-identifying-information-in-a-compliant-and-safe-manner/': 0, 'https://blog.langchain.dev/llms-ac

### Con una peticion a nuestro LLM, trataremos de que despues de analizar el contenido de la pagina, nos devuelva un JSON con los links relevantes

Para esto usaremos un enfoque de `One Shot Prompting`, en donde en el `System Prompt` daremos el ejemplo de un resultado esperado, con estas dos caracteristicas
    - Que todos los links relativos, como `/about`, se les agregue el base path de dicha web para tener el link completo
    - Que el resultado vuelva en formato de JSON

In [27]:
# link_system_prompt = "You are provided with a list of links found on a webpage. \
# You are able to decide which of the links would be most relevant to include in a brochure about the company, \
# such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
# link_system_prompt += "You should respond in JSON as in this example:"
# link_system_prompt += """
# {
#     "links": [
#         {"type": "about page", "url": "https://full.url/goes/here/about"},
#         {"type": "careers page": "url": "https://another.full.url/careers"}
#     ]
# }
# """

In [28]:
# def get_links_user_prompt(website):
#     user_prompt = f"Here is the list of links on the website of {website.url} - "
#     user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
# Do not include Terms of Service, Privacy, email links.\n"
#     user_prompt += "Links (some might be relative links):\n"
#     user_prompt += "\n".join(website.links)
#     return user_prompt

In [29]:
# # verificando formato
# print(link_system_prompt)
# print(get_links_user_prompt(langcahin))

In [30]:
# def filter_relevant_links_with_LLM(url):
#     website = Website(url)
#     messages=[
#             {"role": "system", "content": link_system_prompt},
#             {"role": "user", "content": website.links}
#     ]
#     result = model.invoke(messages)
#     return print(result)

In [31]:
# get_links("https://python.langchain.com/docs/introduction/")

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [32]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = filter_relevant_links(url)
    print("Found links:", links)
    for link in links:
        
        result += f"{link}\n"
        result += Website(link).get_contents()
    return result

# def get_all_details(url):
#     result = "Landing page:\n"
#     result += Website(url).get_contents()
#     links = get_links(url)
#     print("Found links:", links)
#     for link in links["links"]:
#         result += f"\n\n{link['type']}\n"
#         result += Website(link["url"]).get_contents()
#     return result

In [33]:
print(get_all_details("https://www.langchain.com"))

from 94 urls
{'/': 0, '/langchain': 0, '/langsmith': 0, '/langgraph': 0, '/retrieval': 0, '/agents': 0, '/evaluation': 0, 'https://blog.langchain.dev/': 0, '/customers': 0, 'https://academy.langchain.com/': 0, '/community': 0, '/experts': 0, 'https://changelog.langchain.com/': 0, '/testing-guide-ebook': 0, '/stateofaiagents': 0, '/breakoutagents': 0, 'https://python.langchain.com/docs/introduction/': 0, 'https://docs.smith.langchain.com/': 0, 'https://langchain-ai.github.io/langgraph/tutorials/introduction/': 0, 'https://js.langchain.com/docs/introduction/': 0, 'https://langchain-ai.github.io/langgraphjs/tutorials/quickstart/': 0, '/about': 1, '/careers': 1, '/pricing-langsmith': 0, '/pricing-langgraph-platform': 0, '/contact-sales': 1, 'https://smith.langchain.com/': 0, 'https://blog.langchain.dev/ally-financial-collaborates-with-langchain-to-deliver-critical-coding-module-to-mask-personal-identifying-information-in-a-compliant-and-safe-manner/': 0, 'https://blog.langchain.dev/llms-ac

MissingSchema: Invalid URL '/about': No scheme supplied. Perhaps you meant https:///about?

In [ ]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [ ]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [ ]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

In [ ]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [ ]:
create_brochure("HuggingFace", "https://huggingface.com")

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [ ]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [ ]:
stream_brochure("HuggingFace", "https://huggingface.co")

In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 2 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>